### TFM: De Neo4j a Power BI: Integración de Datos para la trazabilidad y análisis de la industria médica

El presente notebook contiene el código en Python desarrollado para la realización del
Análisis Exploratorio de Datos (EDA) y el análisis técnico del Trabajo Fin de Máster (TFM).

### Nota sobre confidencialidad y anonimización

Debido a que los datos utilizados en este trabajo provienen de un entorno industrial real,
no es posible compartir información real, confidencial ni propietaria de ninguna empresa.
Por este motivo, los datos, nombres de variables, identificadores de productos, procesos
y cualquier referencia sensible han sido debidamente anonimizados o agregados con fines
exclusivamente académicos.

El objetivo de este código es demostrar la metodología, la lógica de análisis,
las técnicas empleadas y la estructura del flujo de datos, sin comprometer la
confidencialidad de la información original.

En consecuencia, los archivos de datos originales no se incluyen en este repositorio.
El énfasis del trabajo se centra en el diseño del análisis, la ingeniería de datos
y la interpretación de resultados, y no en la divulgación del conjunto de datos utilizado.


## Proceso ETL y modelado en Neo4j

El proceso de Extracción, Transformación y Carga (ETL) constituye la base del presente
trabajo, permitiendo integrar información proveniente de distintas fuentes operativas
en un modelo de datos orientado a grafos.

Dado el volumen y la complejidad del pipeline implementado, en este notebook se presenta
una visión de alto nivel del proceso ETL, centrándose en su lógica general y en el modelo
conceptual de datos, sin entrar en detalles de implementación específicos.

El objetivo del ETL es transformar datos heterogéneos en una estructura de grafo en Neo4j,
donde entidades como partes, procesos, defectos y datapoints quedan conectadas mediante
relaciones que permiten realizar análisis exploratorios y consultas complejas de forma
eficiente.

Este enfoque facilita la trazabilidad, el análisis de relaciones causa-efecto y la
exploración de patrones que no son evidentes en modelos tabulares tradicionales.


# Orquestación ETL con Airflow (DAG)

Para poblar el grafo, se implementó un DAG en Apache Airflow que ejecuta diariamente un flujo ETL. El DAG separa el proceso en tres etapas principales:
(1) Extract: consulta fuentes relacionales ( Data Warehouses) mediante SQL y genera DataFrames en pandas.
(2) Transform (Clean): estandariza tipos, fechas, nulos, duplicados y aplica reglas de negocio necesarias (por ejemplo, agrupar transacciones por lote y sumar cantidades).
(3) Load: construye el modelo en Neo4j creando/actualizando nodos y relaciones mediante Cypher en batch (UNWIND), asegurando idempotencia con MERGE.

En producción, credenciales y endpoints se administran con prácticas de IT (secrets/vault y variables seguras de Airflow). Para fines académicos, se muestran variables neutrales y se omiten identificadores reales.

In [ ]:
import pandas as pd
from neo4j import GraphDatabase

NEO4J_URI = "neo4j+s://<ANON_HOST>:7687"
NEO4J_USER = "<ANON_USER>"
NEO4J_PASSWORD = "<ANON_PASSWORD>"

# ============================================================
# ETL (Extract – Clean – Transform/Load) hacia Neo4j
# ============================================================
"""
Resumen ETL del pipeline:

1) EXTRACT:
   - Se consulta un origen relacional (p. ej., MES/ERP/DWH en SQL) y se carga en pandas.

2) CLEAN:
   - Se estandarizan fechas, tipos, nulos, duplicados y reglas básicas de negocio.

3) TRANSFORM/LOAD:
   - Se construyen nodos y relaciones en Neo4j usando Cypher con UNWIND para cargar en batch.
"""

def extract_sql_to_df(sql_query: str, sql_conn) -> pd.DataFrame:
    """EXTRACT: SQL -> pandas (conexión gestionada fuera)."""
    return pd.read_sql(sql_query, con=sql_conn)

def clean_df(df: pd.DataFrame) -> pd.DataFrame:
    """CLEAN: limpieza genérica en pandas."""
    df = df.copy()
    if "event_time" in df.columns:
        df["event_time"] = pd.to_datetime(df["event_time"], errors="coerce").dt.strftime("%Y-%m-%dT%H:%M:%S")
    if "quantity" in df.columns:
        df["quantity"] = pd.to_numeric(df["quantity"], errors="coerce").fillna(0)
    return df.drop_duplicates()

def load_df_to_neo4j(df: pd.DataFrame):
    """TRANSFORM/LOAD: pandas -> Neo4j con UNWIND."""
    cypher = """
    UNWIND $rows AS row

    MERGE (p:Part {batch_id: toString(row.batch_id)})
    ON CREATE SET p.material_id = toString(row.material_id)

    MERGE (s:Scrap {container_batch: toString(row.container_batch), reason_code: toString(row.reason_code)})
    ON CREATE SET s.event_time = datetime(row.event_time),
                  s.quantity   = toInteger(row.quantity)

    MERGE (p)-[:GENERATES]->(s)
    """

    rows = df.to_dict("records")
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    with driver.session() as session:
        session.run(cypher, rows=rows)
    driver.close()

# Ejemplo (comentado):
# sql = "SELECT batch_id, material_id, container_batch, reason_code, event_time, quantity FROM <ANON_TABLE> WHERE ..."
# df_raw = extract_sql_to_df(sql, sql_conn)
# df_clean = clean_df(df_raw)
# load_df_to_neo4j(df_clean)

# ============================================================
# Airflow DAG template (pipeline completo, anonimiz.)
# ============================================================
"""
Este DAG ilustra la orquestación de un pipeline ETL hacia Neo4j.
Se muestra como plantilla (template) y se omiten detalles sensibles.

Estructura típica:
- Task 1: Extract (SQL)
- Task 2: Clean (pandas)
- Task 3: Load (Neo4j)
- Opcional: Score / métricas / validaciones de integridad

Nota:
- Variables/secretos se obtienen desde Airflow Variables/Connections o un Vault (no en el código).
"""

import pendulum
from airflow.decorators import dag, task

@dag(
    schedule_interval="0 8 * * *",
    start_date=pendulum.datetime(2024, 4, 16, tz="America/Costa_Rica"),
    catchup=False,
    tags=["Production", "Data_Science", "Neo4j", "ETL"],
)
def neo4j_etl_pipeline():

    @task
    def extract_sql():
        """
        EXTRACT:
        - Ejecuta queries SQL contra fuentes tabulares (ej. MES/DWH)
        - Devuelve dataframes/raw tables para transformar
        """
        # df_ci = run_query(sql_component_issue)
        # df_tc = run_query(sql_task_completions)
        # df_scrap = run_query(sql_scrap)
        return "raw_extracted_data"

    @task
    def transform_pandas(raw_data):
        """
        TRANSFORM:
        - Limpieza y estandarización (fechas, tipos, nulos)
        - Normalización de llaves (ej. container_batch)
        - Agregaciones necesarias para carga eficiente
        """
        # df_clean = clean_and_group(raw_data)
        return "clean_transformed_data"

    @task
    def load_neo4j(clean_data):
        """
        LOAD:
        - Crea/actualiza nodos y relaciones en Neo4j (Cypher + MERGE)
        - Cargas por lotes (batching) para performance
        """
        # write_nodes_and_rels(clean_data)
        return "done"

    raw = extract_sql()
    clean = transform_pandas(raw)
    load_neo4j(clean)

dag = neo4j_etl_pipeline()

## Extracción de datos de neo4j

En este TFM se utiliza **Neo4j**, una base de datos orientada a grafos (graph database).  
A diferencia de una base relacional (tablas), en Neo4j la información se representa mediante:

- **Nodos**: entidades (por ejemplo, `part`, `process`, `scrap`, `datapoint`)
- **Relaciones**: conexiones entre entidades (por ejemplo, una parte *genera* scrap, o un datapoint *pertenece* a una parte)

Para consultar y recorrer el grafo se emplea **Cypher**, el lenguaje de consultas de Neo4j.  
Cypher permite:
- filtrar nodos por propiedades (ej. `scrap_reason_code = 'HP12'`)
- recorrer relaciones para unir información de diferentes entidades (traversals)
- seleccionar variables relevantes para el análisis (EDA y validación de hipótesis)

A continuación se documentan las consultas Cypher utilizadas para extraer subconjuntos de datos desde Neo4j, los cuales alimentan el análisis exploratorio y las visualizaciones del estudio.

Neo4j ofrece una version de escritorio donde se puede ejecutar las consultas o "queries" directamente en el gráfico para este notebook, las consultas se hicieron através de python por lo tanto se importa la librería de neo4j y se crea una variable de gráfico, como es anónimo las credenciales aquí colocadas en la práctica son usuarios y contraseñas, y la dirección IP del cloud donde el gráfico esta en producción. Por motivos de confidencialidad, los identificadores de conexión y las credenciales han sido sustituidos por nombres de variables neutrales, sin exponer información real del entorno productivo.

In [ ]:
from neo4j import GraphDatabase


NEO4J_URI = "neo4j+s://<ANON_HOST>:7687"
NEO4J_USER = "<ANON_USER>"
NEO4J_PASSWORD = "<ANON_PASSWORD>"

graph = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# --- Query 1: Scrap por código de defecto (tendencia temporal)
query1 = """
MATCH (s:scrap)
WHERE s.scrap_reason_code = $scrap_code
RETURN s.quantity, s.scrap_date, s.container_batch
ORDER BY s.scrap_date ASC
"""

# --- Query 2: Bond test + part + scrap para lotes específicos
query2 = """
MATCH (dp:dp_bondtest)-[]->(p:part)-[]->(s:scrap)
MATCH (pr:process)-[]->(p:part)
WHERE dp.datapoint_name = $datapoint_name
  AND s.scrap_reason_code = $scrap_code
  AND s.container_batch IN $batches
RETURN dp.data_value, dp.submitter_name, s.scrap_date, s.quantity, s.container_batch,
       pr.workstation, pr.tasklist_name
ORDER BY s.scrap_date
"""

# --- Query 3: Defectos asociados a materiales específicos
query3 = """
MATCH (p:part)-[:GENERATES]->(s:scrap)
WHERE s.scrap_reason_code = $scrap_code
  AND p.material_id IN $materials
RETURN p.batch_id AS batch,
       p.material_id AS material,
       s.scrap_date AS scrap_date,
       s.quantity AS scrap_qty
"""

params_common = {"scrap_code": "HP12"}

with graph.session() as session:
    df1 = session.run(query1, **params_common).to_data_frame()

    df2 = session.run(
        query2,
        **params_common,
        datapoint_name="Valor bond test ?*",
        batches=["loteA", "loteB", "loteC"]
    ).to_data_frame()

    df3 = session.run(
        query3,
        **params_common,
        materials=["mat_a_01", "mat_a_02", "mat_b_01", "mat_b_02",
                   "mat_c_01", "mat_c_02", "mat_d_01", "mat_d_02"]
    ).to_data_frame()

graph.close()



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Leer el conjunto de datos exportado desde Neo4j
tfm1 = pd.read_csv(r"./data/export_tfm.csv")
tfm1

In [ ]:
# Agrupamiento de los datos

tfm1['s.scrap_date'] = pd.to_datetime(tfm1['s.scrap_date'])
tfm1 = tfm1.sort_values('s.scrap_date')
daily = tfm1.groupby(tfm1['s.scrap_date'].dt.date)['s.quantity'].sum().reset_index()
tfm1['s.quantity'] = tfm1['s.quantity'] * -1

plt.figure(figsize=(12,6))
plt.plot(daily['s.scrap_date'], daily['s.quantity'], marker='o')
plt.title('Tendencia del defecto DEFECT_1 a lo largo del tiempo')
plt.xlabel('Fecha')
plt.ylabel('Cantidad de srap diario')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
#Lectura del conjunto de datos exportado desde Neo4j consulta 2
tfm2 = pd.read_csv(r"./data/export_tfm2.csv")
tfm2

In [ ]:
# Asegurar el orden de las fechas y se convertir a formato numérico para análisis adicional
tfm2['s.scrap_date'] = pd.to_datetime(tfm2['s.scrap_date'])
tfm2 = tfm2.sort_values('s.scrap_date')
tfm2['time_numeric'] = tfm2['s.scrap_date'].apply(lambda x: x.toordinal())
tfm2


In [ ]:
#Gráfica de la tendencia del bond test a lo largo del tiempo
plt.figure(figsize=(12,6))
plt.plot(tfm2['s.scrap_date'], tfm2['dp.data_value'], marker='.', alpha=0.6)
plt.title("Tendencia de la fuerza de adhesión lo largo del tiempo")
plt.xlabel("Fecha")
plt.ylabel("Valor de Bond Test")
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:

# ===============================
# 1. Cargar CSV
# ===============================
tfm2 = pd.read_csv(r"./data/export_tfm2.csv")

# ===============================
# 2. Convertir fecha
# ===============================
tfm2['s.scrap_date'] = pd.to_datetime(tfm2['s.scrap_date'], errors='coerce')

# ===============================
# 3. LIMPIEZA COMPLETA DE dp.data_value
# ===============================

# Convertir todo a string
tfm2['dp.data_value'] = tfm2['dp.data_value'].astype(str)

# Mantener solo números y puntos
tfm2['dp.data_value'] = tfm2['dp.data_value'].str.replace(r'[^0-9.]', '', regex=True)

# Colapsar múltiples puntos ".." o "...." → "."
tfm2['dp.data_value'] = tfm2['dp.data_value'].str.replace(r'\.+', '.', regex=True)

# Eliminar puntos al inicio o final
tfm2['dp.data_value'] = tfm2['dp.data_value'].str.strip('.')

# Convertir a número real
tfm2['dp.data_value'] = pd.to_numeric(tfm2['dp.data_value'], errors='coerce')

# ===============================
# 4. Eliminar outliers imposibles
# ===============================
# Valores reales del Bond Test están entre ~4 y 6
tfm2 = tfm2[(tfm2['dp.data_value'] >= 0) & (tfm2['dp.data_value'] <= 20)]

# ===============================
# 5. Ordenar por fecha
# ===============================
tfm2 = tfm2.sort_values('s.scrap_date')

# ===============================
# 6. Agrupar por día y sacar promedio
# ===============================
daily_bond = (
    tfm2
    .groupby(tfm2['s.scrap_date'].dt.date)['dp.data_value']
    .mean()
    .reset_index()
)

# ===============================
# 7. Graficar tendencia diaria
# ===============================
plt.figure(figsize=(12,6))
plt.plot(daily_bond['s.scrap_date'], daily_bond['dp.data_value'], marker='o', alpha=0.7)
plt.title("Tendencia diaria del Fuerza de Adhesión a lo largo del tiempo")
plt.xlabel("Fecha")
plt.ylabel("Promedio diario de la fuerza de Adhesión")
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import pandas as pd

# ==============================================
# 1. Cargar CSV
# ==============================================
tfm2 = pd.read_csv(r"./data/export_tfm2.csv")

# ==============================================
# 2. Convertir fecha
# ==============================================
tfm2['s.scrap_date'] = pd.to_datetime(tfm2['s.scrap_date'], errors='coerce')

# ==============================================
# 3. LIMPIEZA COMPLETA DE dp.data_value
# ==============================================

# Convertir a string
tfm2['dp.data_value'] = tfm2['dp.data_value'].astype(str)

# Mantener solo números y puntos
tfm2['dp.data_value'] = tfm2['dp.data_value'].str.replace(r'[^0-9.]', '', regex=True)

# Colapsar múltiples puntos
tfm2['dp.data_value'] = tfm2['dp.data_value'].str.replace(r'\.+', '.', regex=True)

# Quitar puntos al inicio o final
tfm2['dp.data_value'] = tfm2['dp.data_value'].str.strip('.')

# Convertir a número
tfm2['dp.data_value'] = pd.to_numeric(tfm2['dp.data_value'], errors='coerce')

# ==============================================
# 4. Eliminar outliers físicamente imposibles
# ==============================================
tfm2 = tfm2[(tfm2['dp.data_value'] >= 0) & (tfm2['dp.data_value'] <= 20)]

# ==============================================
# 5. ANONIMIZAR OPERADORES
# ==============================================

unique_ops = tfm2['dp.submitter_name'].unique()
anon_map = {op: f"Operador_{i+1}" for i, op in enumerate(unique_ops)}

tfm2['submitter_anon'] = tfm2['dp.submitter_name'].map(anon_map)

# ==============================================
# 6. GRAFICO: BOX PLOT POR OPERADOR ANÓNIMO
# ==============================================

plt.figure(figsize=(16,8))
sns.boxplot(
    data=tfm2,
    x='submitter_anon',
    y='dp.data_value'
)

plt.title("Distribución de la Fuerza de Adhesión por Operador (Anonimizado)\nCasos asociados a Scrap DEFECT_1")
plt.xlabel("Operador")
plt.ylabel("Valor de la fuerza de adhesión")
plt.xticks(rotation=90)
plt.grid(axis='y')
plt.tight_layout()
plt.show()


# Reemplazar en el dataframe
tfm2['submitter_anon'] = tfm2['dp.submitter_name'].map(anon_map)


In [ ]:
from scipy.stats import f_oneway

# Crear listas de valores por operador
groups = [group["dp.data_value"].values 
          for name, group in tfm2.groupby("submitter_anon") 
          if len(group) > 1]  # solo operadores con más de 1 dato para evitar errores

# ANOVA de una vía
anova_result = f_oneway(*groups)

anova_result


print("ANOVA F-statistic:", anova_result.statistic)
print("ANOVA p-value:", anova_result.pvalue)


plt.hist(tfm2['dp.data_value'], bins=30)
plt.title("Histograma de Valores de Fuerza de Adhesión")
plt.xlabel("Valor de Fuerza de Adhesión")
plt.ylabel("Frecuencia")
plt.show()

In [ ]:
import pandas as pd
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib.pyplot as plt

# Tukey HSD: requiere que NO haya NaN
df_clean = tfm2[['dp.data_value', 'submitter_anon']].dropna()

# Ejecutar Tukey
tukey_result = pairwise_tukeyhsd(
    endog=df_clean['dp.data_value'],   # valores de Bond Test
    groups=df_clean['submitter_anon'], # los operadores anonimizados
    alpha=0.05
)

print(tukey_result)


In [ ]:
import matplotlib.pyplot as plt

fig = tukey_result.plot_simultaneous(figsize=(14, 10))

ax = plt.gca()

# Cambiar color de líneas y puntos
for line in ax.lines:
    line.set_color("#4DA3D9")

# Cambiar color de textos (opcional, más prolijo)
ax.title.set_color("#374151")
ax.xaxis.label.set_color("#374151")
ax.yaxis.label.set_color("#374151")

plt.title("Comparaciones Post-Hoc Tukey HSD entre Operadores")
plt.xlabel("Media de la fuerza de adhesión")

plt.show()


In [ ]:
# relacion entre la fuerza de adhesion  y el scrap en cantidad absoluta

import matplotlib.pyplot as plt

# Crear columna de scrap absoluto
tfm2["scrap_abs"] = tfm2["s.quantity"].abs()

plt.figure(figsize=(10, 6))

plt.scatter(
    tfm2["dp.data_value"],
    tfm2["scrap_abs"],
    color="#4DA3D9",
    alpha=0.7
)

plt.xlabel("Data Value (dp.data_value)", color="#374151")
plt.ylabel("Cantidad de Scrap (valor absoluto)", color="#374151")
plt.title("Relación entre Data Value y Cantidad de Scrap", color="#374151")

plt.grid(color="#E5E7EB")
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ================================================
# 1. Limpieza básica: scrap absoluto
# ================================================
tfm2['scrap_abs'] = tfm2['s.quantity'].abs()

# Asegurar que la fecha es datetime
tfm2['s.scrap_date'] = pd.to_datetime(tfm2['s.scrap_date'], errors='coerce')

# ================================================
# 2. Consolidar SCRAP REAL por batch (max)
#    y la fecha asociada al scrap (min o max)
# ================================================
scrap_fecha = (
    tfm2.groupby('s.container_batch')
        .agg({
            'scrap_abs': 'max',       # cantidad REAL de scrap por batch
            's.scrap_date': 'min'     # primera fecha del scrap en ese batch
        })
        .sort_values('scrap_abs', ascending=False)
)

print("Scrap consolidado por batch:")
print(scrap_fecha.head(20))

# ================================================
# 3. Top 20 batches
# ================================================
top20 = scrap_fecha.head(20)

# ================================================
# 4. Gráfico de barras: Top 20
# ================================================
plt.figure(figsize=(14,7))
sns.barplot(
    x=top20.index.astype(str),
    y=top20['scrap_abs'],
    palette='Blues_r'
)
plt.title("Top 20 Batches con Mayor Scrap DEFECT_1 (Valor Único por Batch)", fontsize=16)
plt.xlabel("Batch", fontsize=14)
plt.ylabel("Cantidad de Scrap DEFECT_1", fontsize=14)
plt.xticks(rotation=90)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# ================================================
# 5. Scatter Fecha vs Scrap (bivariado)
# ================================================
plt.figure(figsize=(14,7))
plt.scatter(scrap_fecha['s.scrap_date'], scrap_fecha['scrap_abs'], alpha=0.7)
plt.title("Relación entre Fecha y Scrap DEFECT_1 por Batch", fontsize=16)
plt.xlabel("Fecha del Scrap", fontsize=14)
plt.ylabel("Scrap DEFECT_1 (valor único por batch)", fontsize=14)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# ================================================
# 6. Timeline estilo "stem plot"
# ================================================
plt.figure(figsize=(14,6))
plt.stem(scrap_fecha['s.scrap_date'], scrap_fecha['scrap_abs'], basefmt=" ")
plt.title("Timeline de Scrap DEFECT_1 por Batch", fontsize=16)
plt.xlabel("Fecha", fontsize=14)
plt.ylabel("Scrap DEFECT_1", fontsize=14)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# -------------------------------------------------------------------
# 1) Preparar datos
# -------------------------------------------------------------------

# Asegurar que scrap_date es datetime
tfm2['s.scrap_date'] = pd.to_datetime(tfm2['s.scrap_date'])

# Agrupar por batch: scrap total (abs sum) + fecha mínima del batch
batch_group = (
    tfm2.groupby('s.container_batch')
        .agg({
            's.quantity': lambda x: x.abs().sum(),
            's.scrap_date': 'min'
        })
        .reset_index()
)

# Crear columna mes-año
batch_group['mes_anio'] = batch_group['s.scrap_date'].dt.to_period('M').astype(str)

# Seleccionar Top 20 batches
top20 = batch_group.sort_values(by='s.quantity', ascending=False).head(20)

# -------------------------------------------------------------------
# 2) GRAFICAR
# -------------------------------------------------------------------

plt.figure(figsize=(18, 7))

# Barras
bars = plt.bar(
    top20['s.container_batch'].astype(str),
    top20['s.quantity'],
    color=plt.cm.Blues_r(range(20))
)

plt.title("Top 20 Lotes con Mayor Scrap DEFECT_1 (con mes y año del lote)", fontsize=18)
plt.xlabel("Lote", fontsize=14)
plt.ylabel("Cantidad de Scrap DEFECT_1", fontsize=14)
plt.xticks(rotation=90, fontsize=10)

# -------------------------------------------------------------------
# 3) Insertar fecha dentro de cada barra (centrado)
# -------------------------------------------------------------------

for bar, fecha in zip(bars, top20['mes_anio']):
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,   # centro horizontal
        height / 2,                          # centro vertical
        fecha,
        ha='center',
        va='center',
        rotation=90,
        fontsize=10,
        fontweight='bold',
        color='white'                        # Contraste
    )

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Copia del dataframe original para no dañarlo
df_turno = tfm2.copy()

# --- 1. Convertir scrap_date a datetime
df_turno['s.scrap_date'] = pd.to_datetime(df_turno['s.scrap_date'])

# --- 2. Corregir scrap a positivo
df_turno['scrap_qty'] = df_turno['s.quantity'].abs()

# --- 3. Definir turnos
def asignar_turno(hora):
    if hora >= 6 and hora < 15.5:     # 6:00 — 15:29
        return 'A'
    elif hora >= 15.5 and hora < 22:  # 15:30 — 21:59
        return 'B'
    else:                             # 22:00 — 5:59
        return 'C'

df_turno['turno'] = df_turno['s.scrap_date'].dt.hour.apply(asignar_turno)

# --- 4. Quedarnos solo con el valor máximo de scrap por batch (evita duplicados)
df_batch_max = (
    df_turno.groupby('s.container_batch')['scrap_qty']
            .max()
            .reset_index()
)

# Ahora necesitamos unir esto de nuevo para recuperar el turno:
df_merged = df_batch_max.merge(
    df_turno[['s.container_batch', 'turno']].drop_duplicates(),
    on='s.container_batch',
    how='left'
)

# --- 5. Sumar el scrap total por turno
scrap_por_turno = df_merged.groupby('turno')['scrap_qty'].sum().reset_index()

# --- 6. Graficar
plt.figure(figsize=(8,6))
plt.bar(scrap_por_turno['turno'], scrap_por_turno['scrap_qty'], color=['#4DA3D9', '#1F6FA8', '#A9D6F5'])

plt.title('Scrap DEFECT_1 por Turno (con valores positivos y agrupado por Lote)')
plt.xlabel('Turno')
plt.ylabel('Cantidad total de Scrap DEFECT_1')

# Mostrar valores arriba de cada barra
for i, v in enumerate(scrap_por_turno['scrap_qty']):
    plt.text(i, v + 50, str(int(v)), ha='center', fontweight='bold')

plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Copia del dataframe original
df_box = tfm2.copy()

# --- 1. Convertir scrap_date a datetime
df_box['s.scrap_date'] = pd.to_datetime(df_box['s.scrap_date'])

# --- 2. Asegurar que Bond Test sea numérico
df_box['dp.data_value'] = pd.to_numeric(df_box['dp.data_value'], errors='coerce')

# --- 3. Definir turnos por hora
def asignar_turno(hora):
    if 6 <= hora < 15.5:
        return 'A'
    elif 15.5 <= hora < 22:
        return 'B'
    else:
        return 'C'

df_box['turno'] = df_box['s.scrap_date'].dt.hour.apply(asignar_turno)

# --- 4. Crear figura con 3 boxplots (uno por turno)
plt.figure(figsize=(14,6))

turnos = ['A', 'B', 'C']

for i, t in enumerate(turnos):
    plt.subplot(1, 3, i+1)
    sns.boxplot(
        data=df_box[df_box['turno'] == t],
        y='dp.data_value',
        color=['#4DA3D9', '#1F6FA8', '#A9D6F5'][i]
    )
    plt.title(f'Turno {t}')
    plt.ylabel('Prueba de adhesión (lbf)')
    plt.ylim(df_box['dp.data_value'].min() - 0.1,
             df_box['dp.data_value'].max() + 0.1)

plt.suptitle("Distribución de la prueba de Adhesión por Turno (Scrap DEFECT_1)", fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
from scipy.stats import f_oneway

# Extraer valores por turno
bond_A = df_box[df_box['turno'] == 'A']['dp.data_value'].dropna()
bond_B = df_box[df_box['turno'] == 'B']['dp.data_value'].dropna()
bond_C = df_box[df_box['turno'] == 'C']['dp.data_value'].dropna()

# ANOVA de un factor
F_stat, p_value = f_oneway(bond_A, bond_B, bond_C)

print("ANOVA por Turno")
print("F-statistic:", F_stat)
print("p-value:", p_value)


In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import pandas as pd

# Crear dataframe para Tukey
df_anova_turnos = df_box[['dp.data_value', 'turno']].dropna()

# Tukey HSD
tukey_turnos = pairwise_tukeyhsd(
    endog=df_anova_turnos['dp.data_value'],
    groups=df_anova_turnos['turno'],
    alpha=0.05
)

print(tukey_turnos)


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Preparamos datos
df_anova_turnos = df_box[['dp.data_value', 'turno']].dropna()

# Tukey HSD
tukey_turnos = pairwise_tukeyhsd(
    endog=df_anova_turnos['dp.data_value'],
    groups=df_anova_turnos['turno'],
    alpha=0.05
)

# Convertir resultados a dataframe
tukey_df = pd.DataFrame(data=tukey_turnos._results_table.data[1:], 
                        columns=tukey_turnos._results_table.data[0])

# Crear columnas numéricas
tukey_df['diff'] = tukey_df['meandiff'].astype(float)
tukey_df['lower'] = tukey_df['lower'].astype(float)
tukey_df['upper'] = tukey_df['upper'].astype(float)

# --- Gráfico ---
plt.figure(figsize=(10,6))

# Dibujar intervalos
for i in range(len(tukey_df)):
    plt.plot([tukey_df['lower'][i], tukey_df['upper'][i]], [i, i], 'k-', lw=2)
    plt.plot(tukey_df['diff'][i], i, 'o', color='blue')

# Línea vertical en 0
plt.axvline(0, color='red', linestyle='--')

plt.yticks(range(len(tukey_df)), tukey_df['group1'] + " vs " + tukey_df['group2'])
plt.xlabel("Diferencia de medias (Fuerza de Adhesión)")
plt.title("Tukey HSD – Comparación entre Turnos")
plt.grid(axis='x', linestyle='--', alpha=0.5)

plt.show()


Analisis bi variable

In [ ]:
tfm2['scrap_abs'] = tfm2['s.quantity'].abs()

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
sns.scatterplot(data=tfm2, x="dp.data_value", y="scrap_abs", alpha=0.5)

plt.title("Relación entre Fuerza de Adhesión y Cantidad de Scrap DEFECT_1")
plt.xlabel("Fuerza de Adhesión")
plt.ylabel("Scrap DEFECT_1 (cantidad absoluta)")
plt.grid(True)
plt.show()

tfm2[['dp.data_value', 'scrap_abs']].corr()


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression

# --- Cargar archivo ---
df_tfm3 = pd.read_csv(r"./data/export_tfm2.csv")

# --- Limpieza de datos ---
# Convertir bond test a float
df_tfm3['dp.data_value'] = pd.to_numeric(df_tfm3['dp.data_value'], errors='coerce')

# Scrap absoluto
df_tfm3['s.quantity_abs'] = df_tfm3['s.quantity'].abs()

# Eliminar filas con valores faltantes (muy importante)
df_clean = df_tfm3.dropna(subset=['dp.data_value', 's.quantity_abs'])

# --- 1. Correlación de Pearson ---
pearson_corr, pearson_p = pearsonr(df_clean['dp.data_value'], df_clean['s.quantity_abs'])

# --- 2. R² usando regresión lineal ---
X = df_clean[['dp.data_value']]
y = df_clean['s.quantity_abs']

model = LinearRegression()
model.fit(X, y)
r2 = model.score(X, y)

# --- Imprimir resultados ---
print("Correlación de Pearson:", round(pearson_corr, 4))
print("P-value de Pearson:", pearson_p)
print("R² del modelo lineal:", round(r2, 4))


In [ ]:
# Asegurar dt
df_tfm3['s.scrap_date'] = pd.to_datetime(df_tfm3['s.scrap_date'])
df_tfm3['year_month'] = df_tfm3['s.scrap_date'].dt.to_period('M').astype(str)

# Scrap absoluto
df_tfm3['s.quantity_abs'] = df_tfm3['s.quantity'].abs()

# Scrap total por mes
scrap_mensual = df_tfm3.groupby('year_month')['s.quantity_abs'].sum()

# Mes con más scrap
mes_max = scrap_mensual.idxmax()
mes_max


In [ ]:
df_mes_critico = df_tfm3[df_tfm3['year_month'] == mes_max].copy()
df_mes_critico.head()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# --- 1. Preparar dataframe ---
df_tfm3['s.scrap_date'] = pd.to_datetime(df_tfm3['s.scrap_date'])
df_tfm3['s.quantity_abs'] = df_tfm3['s.quantity'].abs()

# --- 2. Extraer mes crítico ---
df_sep = df_tfm3[df_tfm3['s.scrap_date'].dt.to_period('M') == '2024-09'].copy()

# --- 3. Crear columnas de día y hora ---
df_sep['Fecha'] = df_sep['s.scrap_date'].dt.date
df_sep['Hora'] = df_sep['s.scrap_date'].dt.hour

# --- 4. Agrupar correctamente:
#     Para cada día-hora y batch, obtener el scrap máximo
df_grouped = (
    df_sep.groupby(['Fecha', 'Hora', 's.container_batch'])['s.quantity_abs']
    .max()   # ← EVITA la multiplicación x5
    .reset_index()
)

# --- 5. Ahora agrupar por Fecha y Hora sumando LOTES únicos ---
df_heat = (
    df_grouped.groupby(['Fecha', 'Hora'])['s.quantity_abs']
    .sum()   # suma real por hora sin duplicados
    .unstack(fill_value=0)
)

# --- 6. Plot del heatmap ---


plt.figure(figsize=(16, 9))

sns.heatmap(
    df_heat,
    cmap='Blues',
    linewidths=0.3,
    linecolor='#E5E7EB',
    cbar_kws={'label': 'Cantidad de Scrap (valor absoluto)'}
)

plt.title(
    "Heatmap de Scrap DEFECT_1 — Mes Crítico 2024-09",
    fontsize=16,
    color="#374151"
)
plt.xlabel("Hora del día", color="#374151")
plt.ylabel("Fecha", color="#374151")

plt.show()



In [ ]:
df_tfm4 = pd.read_csv(r"./data/export_tfm2.csv")
df_tfm4


In [ ]:
import pandas as pd

# --- 1. Vista general ---
print("Shape (filas, columnas):")
print(df_tfm4.shape)
print("\nPrimeras filas:")
display(df_tfm4.head())

# --- 2. Tipos de datos ---
print("\nTipos de datos:")
print(df_tfm4.dtypes)

# --- 3. Descripción estadística de las variables numéricas ---
print("\nDescripción estadística (numéricas):")
display(df_tfm4.describe())

# --- 4. Cantidad total de data_value, min, max, mean ---
print("\nResumen de dp.data_value:")
print("Count  :", df_tfm4['dp.data_value'].count())
print("Min    :", df_tfm4['dp.data_value'].min())
print("Max    :", df_tfm4['dp.data_value'].max())
print("Mean   :", df_tfm4['dp.data_value'].mean())

# --- 5. ¿Cuántos operadores/submitters diferentes hay? ---
print("\nCantidad de submitters únicos:")
print(df_tfm4['dp.submitter_name'].nunique())

print("\nLista de submitters únicos:")
print(df_tfm4['dp.submitter_name'].unique())

# --- 6. ¿Cuántos batches diferentes hay? ---
print("\nCantidad de container_batch únicos:")
print(df_tfm4['s.container_batch'].nunique())

# --- 7. Valores únicos de scrap quantities (por curiosidad EDA) ---
print("\nValores distintos de s.quantity:")
print(df_tfm4['s.quantity'].unique())


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_mat_tfm = pd.read_csv(r"./data/export_tfm3.csv")
df_mat_tfm

In [ ]:
print (df_mat_tfm['material'].unique())


material_map = {
    '808886-01': 'mat_a_01',
    '808886-02': 'mat_a_02',

    '809273-01': 'mat_b_01',
    '809273-02': 'mat_b_02',

    '809297-01': 'mat_c_01',
    '809297-02': 'mat_c_02',

    '809299-01': 'mat_d_01',
    '809299-02': 'mat_d_02'
}

df_mat_tfm['material_anon'] = df_mat_tfm['material'].map(material_map)


In [ ]:

df_mat_tfm['scrap_rate'] = abs(df_mat_tfm['scrap_qty']) / df_mat_tfm['finish_q']


In [ ]:
scrap_rate_by_mat = (
    df_mat_tfm.groupby('material_anon')['scrap_rate']
              .mean()
              .sort_values(ascending=False)
)

scrap_rate_by_mat


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(9,5))
scrap_rate_by_mat.plot(kind='bar')

plt.title("Tasa de fallos por Material Anonimizado")
plt.xlabel("Material")
plt.ylabel("Tasa de Fallos")
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)

plt.show()

In [ ]:
print (df_mat_tfm['process_line'].unique())
line_map = {
    'LINE_A': 'proc_x_01',
    'LINE_B': 'proc_x_02'
}

df_mat_tfm['process_line_anon'] = df_mat_tfm['process_line'].map(line_map)


In [ ]:
scrap_rate_by_line = (
    df_mat_tfm.groupby('process_line_anon')['scrap_rate']
              .mean()
              .sort_values(ascending=False)
)

scrap_rate_by_line


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(7,4))
scrap_rate_by_line.plot(kind='bar', color='steelblue')

plt.title("Tasa de Fallas por Línea de Proceso (Anonimizada)")
plt.xlabel("Línea de Proceso")
plt.ylabel("Tasa de Fallas")
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)

plt.show()

In [ ]:
import scipy.stats as stats

line1 = df_mat_tfm[df_mat_tfm['process_line_anon'] == 'proc_x_01']['scrap_rate']
line2 = df_mat_tfm[df_mat_tfm['process_line_anon'] == 'proc_x_02']['scrap_rate']

t_stat, p_val = stats.ttest_ind(line1, line2, equal_var=False)  # Welch’s t-test
t_stat, p_val



In [ ]:
df_mat_tfm

In [ ]:
df_tfm2= pd.read_csv(r"./data/export_tfm2.csv")

df_tfm2

In [ ]:
df_tfm2 = df_tfm2.rename(columns={'s.container_batch': 'batch'})
df_merged = df_mat_tfm.merge(
    df_tfm2[['batch', 'dp.data_value']],   # solo traemos bond test y batch
    on='batch',
    how='inner'
)



In [ ]:
df_merged.head()
df_merged.columns


In [ ]:
df_merged['dp.data_value'] = pd.to_numeric(df_merged['dp.data_value'], errors='coerce')
df_merged = df_merged[df_merged['dp.data_value'] <= 20]
df_merged = df_merged.dropna(subset=['dp.data_value'])


df_bond_mean = df_merged.groupby(['batch', 'material_anon'], as_index=False)['dp.data_value'].mean()

df_bond_mean.rename(columns={'dp.data_value': 'bond_test_mean'}, inplace=True)

df_bond_mean.head()

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(data=df_bond_mean, x='material_anon', y='bond_test_mean')
plt.xlabel("Material Anonimizado")
plt.ylabel("Fuerza de Adhesión Promedio")
plt.title("Fuerza de Adhesión por Material (Anonimizado)")
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.show()


In [ ]:
groups = [g['bond_test_mean'].values 
          for name, g in df_bond_mean.groupby('material_anon')]

F, p = stats.f_oneway(*groups)
F, p


In [ ]:
import pandas as pd

# --- Turno A/B/C según la hora ---
def asignar_turno(fecha):
    h = fecha.hour
    m = fecha.minute
    t = h*60 + m  # minutos desde medianoche
    
    # Turno A: 6:00–15:30  (360–930)
    if 360 <= t < 930:
        return "A"
    # Turno B: 15:30–22:00 (930–1320)
    elif 930 <= t < 1320:
        return "B"
    # Turno C: 22:00–6:30 (1320–390)
    else:
        return "C"

# --- OrdenTurno para ordenar en Power BI ---
def asignar_orden(turno):
    if turno == "A":
        return 1
    elif turno == "B":
        return 2
    else:
        return 3

# --- PROCESAR EL DATAFRAME ---
df_merged["scrap_date"] = pd.to_datetime(df_merged["scrap_date"])

df_merged["Turno"] = df_merged["scrap_date"].apply(asignar_turno)
df_merged["OrdenTurno"] = df_merged["Turno"].apply(asignar_orden)

print(df_merged[["scrap_date", "Turno", "OrdenTurno"]].head())


In [ ]:
# --- 2. Asegurar que scrap_date es datetime real ---
df_merged["scrap_date"] = pd.to_datetime(df_merged["scrap_date"], errors="coerce")

# --- 3. Convertir scrap_date a minutos desde medianoche ---
df_merged["time_minutes"] = (
    df_merged["scrap_date"].dt.hour * 60 +
    df_merged["scrap_date"].dt.minute
)

# --- 4. Función para asignar Turno sin volver loca a Power BI ---
def clasificar_turno(mins):
    if pd.isna(mins):
        return None

    # Turno A: 6:00–15:30  (360–930)
    if 360 <= mins < 930:
        return "A"
    # Turno B: 15:30–22:00  (930–1320)
    elif 930 <= mins < 1320:
        return "B"
    # Turno C: 22:00–6:30 (1320–390)
    else:
        return "C"

# --- 5. Crear columna Turno ---
df_merged["Turno"] = df_merged["time_minutes"].apply(clasificar_turno)

# --- 6. Crear columna OrdenTurno (para ordenar A-B-C sin errores) ---
orden = {"A": 1, "B": 2, "C": 3}
df_merged["OrdenTurno"] = df_merged["Turno"].map(orden)

# --- 7. (Opcional) Eliminar columna auxiliar ---
df_merged.drop(columns=["time_minutes"], inplace=True)

# --- 8. Exportar listo para Power BI ---
df_merged.to_csv("df_final_para_powerbi.csv", index=False)

print("Archivo listo: df_final_para_powerbi.csv")